# Lesson 1 Demo — From Regression to Deep Learning

This notebook mirrors the Lesson 1 slides: generate a non-linear 2D dataset,
train a small Keras model, and visualize decision regions.

## Setup
If packages are missing, uncomment the pip installs.

In [ ]:
# !pip install -q tensorflow scikit-learn matplotlib
import numpy as np
import matplotlib.pyplot as plt

try:
    from sklearn.datasets import make_circles
    HAVE_SK = True
except Exception:
    HAVE_SK = False

from tensorflow import keras
from tensorflow.keras.layers import Dense


## Make a 2D dataset
We use `make_circles` if available; otherwise a lightweight spiral generator.

In [ ]:
def make_spiral(n=500, noise=0.2, seed=0):
    rng = np.random.default_rng(seed)
    n2 = n // 2
    t = np.linspace(0, 2*np.pi, n2)
    r = np.linspace(0.2, 1.0, n2)
    x1 = np.c_[r*np.cos(t), r*np.sin(t)] + noise * rng.standard_normal((n2, 2))
    x2 = np.c_[-r*np.cos(t), -r*np.sin(t)] + noise * rng.standard_normal((n2, 2))
    Xs = np.vstack([x1, x2]).astype('float32')
    ys = np.r_[np.zeros(n2), np.ones(n2)].astype('float32')
    return Xs, ys

if HAVE_SK:
    X, y = make_circles(n_samples=1000, factor=0.5, noise=0.1, random_state=0)
    X = X.astype('float32'); y = y.astype('float32')
else:
    X, y = make_spiral(n=800, noise=0.15, seed=0)

plt.figure(figsize=(4.5,4))
plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', s=12, edgecolor='k')
plt.title('Dataset preview')
plt.show()


## Build and train a small network
Two hidden layers with ReLU, sigmoid output for binary classification.

In [ ]:
model = keras.Sequential([
    Dense(8, activation='relu', input_shape=(2,)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X, y, batch_size=32, epochs=20, validation_split=0.2, verbose=0)
float(history.history['val_accuracy'][-1])


## Plot decision regions
Compare model predictions across a mesh grid with the data points.

In [ ]:
xx, yy = np.meshgrid(
    np.linspace(X[:,0].min()-0.5, X[:,0].max()+0.5, 200),
    np.linspace(X[:,1].min()-0.5, X[:,1].max()+0.5, 200)
)
grid = np.c_[xx.ravel(), yy.ravel()].astype('float32')
probs = model.predict(grid, verbose=0).reshape(xx.shape)

plt.figure(figsize=(5,4))
cs = plt.contourf(xx, yy, probs, levels=20, cmap='RdBu', alpha=0.6)
plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', edgecolor='k', s=12)
plt.title('Decision regions')
plt.show()
